In [1]:
# THERE ARE SEPARATE NOTEBOOKS FOR VISUALIZATIONS, DATASET ANALYSIS, ETC. IN THE REPO.

import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')

In [ ]:
df.Yield.describe()

In [ ]:
# THIS IS USED WHEN CONNECTING TO COMPUTE NODES TO CHECK IF EVERYTHING IS WORKING

import os

print(os.getcwd())

In [ ]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

variety_dummies = pd.get_dummies(df.Variety)
variety_dummies.columns = [i for i in range (0,174)]
df = pd.concat([df, variety_dummies], axis=1)

In [ ]:
df[df.isnull()]
df.isnull().sum()

In [ ]:
# CURRENTLY NECESSARY IF: USING 8 ADDITIONAL VARIETY COLUMNS (BINARY REPRESENTATION)

UNIQUE_VARIETIES = np.unique(df.Variety)
INDICES = [i for i in range(0, len(UNIQUE_VARIETIES))]
# print(np.where(UNIQUE_VARIETIES=='V000017'))
df.VarietyBin = df.Variety.apply(lambda var: np.where(UNIQUE_VARIETIES==var)[0][0])
# print(UNIQUE_VARIETIES)
# print(df.Variety)

df.VarietyBin = df.VarietyBin.apply(lambda var: '{0:08b}'.format(var))
print("done")
df.VarietyBin = df.VarietyBin.apply(lambda var: list(var))
print("done")
variety_binary_df = pd.DataFrame(pd.Series(df.VarietyBin[i]) for i in range(0, len(df.VarietyBin)))
df = pd.concat([df, variety_binary_df], axis=1)

In [ ]:
# GOAL OF THIS MODULE:
# Encode the planting date as a season

# remove the dates that are "."
df = df[~df['Planting date'].str.match("\.")]
plant_date = df['Planting date'].apply(lambda dt: pd.to_datetime(dt))
plant_months = plant_date.apply(lambda dt: dt.month)
season = plant_date.rename("Season")
season = pd.to_datetime(season)
season = season.apply(lambda dt: (dt.month%12 + 3)//3)
# df['Plant date'] = pd.to_datetime(df['Plant date'])
df = pd.concat([df, season], axis=1)

# plant_date = pd.to_datetime(df['Planting date'], infer_datetime_format=True)
# df = df['Planting date'].apply(lambda dt: (dt.month%12 + 3)//3)
# pd.get_dummies(df['Planting date'])

In [ ]:
# ADD MONTH OF MAY AND JUNE ONE HOT ENCODING INTO THE DATAFRAME
pd.get_dummies(plant_months).sum()
june = pd.get_dummies(plant_months).loc[:,6]
june = june.rename("June")
may = pd.get_dummies(plant_months).loc[:,5]
may = may.rename("May")
df = pd.concat([df, may], axis=1)
df = pd.concat([df, june], axis=1)

In [ ]:
# LATITUDE AND LONGITUDE CLUSTERING INTO FEATURES

from sklearn.cluster import KMeans

latlong = df.loc[:, ['Latitude', 'Longitude']]

kmeans = KMeans(n_clusters=4, random_state=0, n_jobs=-1).fit(latlong)
kmeans.labels_.shape
lat_long_dummies = pd.get_dummies(kmeans.labels_)
lat_long_dummies = lat_long_dummies.rename(index=int, columns={0: "Loc Clust 0",
                                                               1: "Loc Clust 1",
                                                               2: "Loc Clust 2",
                                                               3: "Loc Clust 3"})
df = pd.concat([df, lat_long_dummies], axis = 1)

In [ ]:
df.shape

In [ ]:
#the below 3 nodes were used to generate csv for use with https://www.darrinward.com/lat-long/

from sklearn.cluster import KMeans
latlong = df.loc[:, ['Latitude', 'Longitude']]
kmeans = KMeans(n_clusters=8, random_state=0, n_jobs=-1).fit(latlong)
dummies = pd.get_dummies(kmeans.labels_)
dummies.columns = ["Loc Clust " + str(x) for x in dummies.columns]
reformatted = dummies.idxmax(axis=1)
latlong = pd.concat([latlong, reformatted], axis=1)
df = pd.concat([df, dummies], axis=1)
# dummies = dummies.rename(index=int, )

In [ ]:
import random

colors = ["#%06x" % random.randint(0, 0xFFFFFF) for i in range(0,8)]
colors

In [ ]:
latlong = latlong.rename(index=str, columns={0:"name"})
colors_dict = {key:colors[int(key[-1])] for key in dummies.columns}
latlong['color'] = latlong['name'].map(colors_dict)
# colors.keys = [key for key in dummies.columns]
latlongsample = latlong.sample(frac=0.01)
latlongsample.to_csv("latlongsample.csv")
latlongsample.reindex()
latlong.to_csv("latlongname.csv")
latlongsample.drop(['color'], axis=1).to_csv("latlongsamplenocolor.csv")



In [ ]:
#REMOVE ANY NAN VALUES

print(df.columns)
df = df[~df.Silt.isnull()]
df = df[~df['Loc Clust 1'].isnull()]
df = df[~df['Yield'].isnull()]
# df = df[~df['Loc Clust 0'].isnull()]

In [ ]:
df.isnull().sum()

In [ ]:
for col in df.columns:
    print(col, type(df[col][0]))

In [ ]:
df

In [ ]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

# set if want to drop some columns specifically
should_drop = 1
# columns_to_drop = ['Experiment', 'Location',
#                    'Check Yield', 'Yield difference', 'Latitude',
#                    'Longitude', 'PI', 'Variety', 'Planting date', 'Season']

# BELOW DROP IS USED FOR THE DF_DICT APPROACH
# columns_to_drop = ['Experiment', 'Location',
#                    'Check Yield', 'Yield difference', 
#                    'PI', 'Planting date', 'Season', 'Variety']
# columns_to_drop = ['Experiment', 'Location',
#                  'Check Yield', 'Yield difference', 'PI', 'Planting date', 'Variety']
columns_to_drop = ['Experiment', 'Location',
                 'Check Yield', 'Yield difference', 'PI', 'Planting date', 'Variety', 'Latitude', 'Longitude']

# set if want to keep some columns specifically
should_keep = 0
# columns_to_keep = ['Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3']
columns_to_keep = []

# columns_to_keep_top = ['Silt', 'Precipitation', 'Temperature', 'Solar Radiation', 'Organic matter']
# columns_VARIETIES_ONLY = np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

#set the below variable to whatever columns you want to keep
columns_to_keep = columns_to_keep

MUST_HAVE_COLUMNS = ['Yield']
# print(columns_to_keep)

df = df.drop(columns_to_drop, axis=1) if should_drop else df
df = df.loc[:, np.concatenate((columns_to_keep, MUST_HAVE_COLUMNS))] if should_keep else df
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))
print("The final dataframe has columns: ", df.columns)

In [ ]:
df = pd.concat([df, latlong], axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
df = df.loc[:,'Sand']

In [ ]:
[x for x in range(0,8)]

In [ ]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA

print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
print(df.isnull().sum())
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

In [ ]:
X_train.isnull().sum()

In [ ]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT ( NOT FOR USE WITH DICTIONARY METHOD)
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT ( NOT FOR USE WITH DICTIONARY METHOD)

X = df.drop(['Yield', 'YieldBucket'], axis=1)

# print(X.columns)

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.95, random_state = 42)

INPUT_COLS = X_train.columns
# TEST_COLS = y_train.columns

In [ ]:
# THIS MODULE IS ONLY BEING USED FOR THE COMPARISON OF CUMULATIVE TRAINING WITH DICTIONARY BASED TRAINING
X_train_varieties = X_train.Variety
X_test_varieties = X_test.Variety
X_train = X_train.drop(['Variety'], axis=1)
X_test = X_test.drop(['Variety'], axis=1)

In [ ]:
print("X_train shape:", X_train.shape, "\ny_train shape:", y_train.shape)

In [ ]:
X_train = X_train[~X_train.Temperature.isnull()]

In [ ]:
X_train.isnull().sum()

In [ ]:
for val in X_train.columns:
    print(val)

In [ ]:
# GOAL OF THIS MODULE: PCA OUTPUT AND VISUAL ANALYSIS

from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def gen_PCA(X_train_, target_series, visualize=False):
    # vis_features = ['Silt', 'Precipitation', 'Organic matter', 'Solar Radiation', 'Temperature', 'CEC']
    # X_train_scaled = StandardScaler().fit_transform(X_train_.loc[:, vis_features])
    X_train_scaled = StandardScaler().fit_transform(X_train_)

    pca = PCA(n_components=8)
    principalComponents = pca.fit_transform(X_train_scaled)
#     principal_df = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
    principal_df = pd.DataFrame(data=principalComponents)
    print(principal_df.shape)
#     print(X_train_.shape)
    print(target_series.shape)
    
    # WHY DOES THE BELOW LINE NOT WORK AND INSTEAD I NEED TO USE THE FOLLOWING TWO LINES?
#     final_df = pd.concat([principal_df, target_series], axis=1)
    final_df = principal_df
    final_df['Yield'] = target_series
    
    print(final_df.shape)
    print(final_df)
    if visualize:
        fig = plt.figure(figsize=(20,20))
        ax = fig.add_subplot(1,1,1) 
        ax.set_xlabel('Principal Component 1', fontsize = 15)
        ax.set_ylabel('Principal Component 2', fontsize = 15)
        ax.set_title('2 component PCA', fontsize = 20)
        targets = ['high', 'medium', 'low']
        colors = ['g', 'y', 'r']
        for target, color in zip(targets,colors):
            indicesToKeep = final_df['YieldBucket'] == target
            ax.scatter(final_df.loc[indicesToKeep, 'principal component 1']
                       , final_df.loc[indicesToKeep, 'principal component 2']
                       , c = color
                       , s = 3)
        ax.legend(targets)
        ax.grid()
        fig.savefig("graphs/PCA-bucketing.png")
    return final_df
    
final_df = gen_PCA(X_train, df['Yield'])

In [ ]:
len(X_train.columns)
final_df.isnull().sum()
final_df[final_df.Yield.isnull()]

In [ ]:
final_df = final_df[~final_df.Yield.isnull()]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(['Yield'], axis=1), final_df.loc[:, 'Yield'], train_size = 0.5, test_size = 0.1)

In [ ]:
for i in range(0,8):
    X_train[i] = pd.to_numeric(X_train[i])
    X_test[i] = pd.to_numeric(X_test[i])

In [ ]:
# THIS FUNCTION WILL EVALUATE ERRORS BASED ON RMSE (FROM SYNGENTA CHALLENGE SPEC)
# AND ALSO WILL EVALUATE BASED ON AVERAGE ERROR

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def evaluate_errors(prediction, actual):
    RMSE_error = np.sqrt(mean_squared_error(prediction, actual))
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((prediction - actual) / actual) * 100)
    print("Average Error details:\n", avg_error_vector.describe())
    return avg_error_vector, RMSE_error

import numpy as np

def mean_absolute_percentage_error(y_pred, y_true): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def MAPE(prediction, actual):
    numerator = np.abs((np.subtract(actual, prediction)))
    denominator = prediction
    quotient = np.divide(numerator, denominator)
    summ = np.sum(quotient)
    return summ/len(prediction)
    
# MAPE(preds, y_test)

In [ ]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 20, 30],
#     'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv=2,
                          n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
# MOST USED CLASSIFIER CURRENTLY

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

regr = RandomForestRegressor(min_samples_leaf=3, min_samples_split=12,n_estimators=20, max_depth=20,random_state=32, verbose=1, n_jobs=-1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)
print("MAE", mean_absolute_error(preds, y_test))
print("MAPE", mean_absolute_percentage_error(preds, y_test))
print("RMSE", np.sqrt(mean_squared_error(preds, y_test)))
# evaluate_errors(preds, y_test)

In [ ]:
variety_df = pd.DataFrame({
    'Variety': X_test_varieties,
    'preds': preds,
    'actual': y_test
})

In [ ]:
variety_df.Variety

In [ ]:
df_dict = {variety: variety_df.loc[variety_df.Variety == variety] for variety in UNIQUE_VARIETIES}

In [ ]:
pure_df_dict = {variety: df.loc[df.Variety == variety] for variety in UNIQUE_VARIETIES}

In [ ]:
for key, df in df_dict.items():
    df_dict[key] = df.drop(['Variety'], axis=1)


In [ ]:
RMSE_errs = np.array([])
varieties = np.array([])

def RMSE(preds, y_test):
    return np.sqrt(mean_squared_error(preds, y_test))

for key, df in df_dict.items():
    RMSE_value = RMSE(df.preds, df.actual)
    RMSE_errs = np.append(RMSE_errs, RMSE_value)
    varieties = np.append(varieties, key)

In [ ]:
pd.DataFrame({
    'vars': varieties,
    'errs': RMSE_errs
}).errs.describe()

In [ ]:
for key, df in df_dict.items():
    if(df.shape[0] == 0):
        print(key)
        
del df_dict['V152779']

In [ ]:
from sklearn.tree import export_graphviz
import os

export_graphviz(regr.estimators_[0],
                feature_names=X.columns,
                filled=True,
                rounded=True)

os.system('dot -Tpng tree.dot -o tree.png')


In [ ]:
import sys

orig_stdout = sys.stdout
f = open('classifier_out.txt', 'w')
sys.stdout = f

tree_to_code(regr.estimators_[0], feature_names=X_train.columns)

sys.stdout = orig_stdout
f.close()


In [ ]:
X_train.columns = X_train.columns.astype(str)

In [ ]:
# GET OUTPUT OF FEATURE IMPORTANCE

def get_feature_importances(regr):
    feature_importances = regr.feature_importances_
    feature_importances = pd.Series(feature_importances)
    feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
    for index, row in feature_importance_df.iterrows():
#         print(row['feature'], 'has importance: ', row['feature_importance'])
          print(row['feature'], ",", row['feature_importance'])
get_feature_importances(regr)

In [ ]:
np.mean(np.abs(np.divide(np.subtract(y_test, preds), y_test)))
np.sum(np.abs(np.subtract(y_test, preds))) / len(y_test)

In [ ]:
# MODELS LIGHTGBM

from lightgbm import LGBMRegressor

clf = LGBMRegressor(n_estimators=100, max_depth=25, learning_rate=0.1, n_jobs = -1, boosting_type='dart')
# clf.fit(X_train, np.log1p(y_train))
clf.fit(X_train, y_train)
# preds = np.expm1(clf.predict(X_test))
preds = clf.predict(X_test)

preds = pd.DataFrame(preds)
y_test = pd.DataFrame(y_test)

from sklearn.metrics import mean_absolute_error
print("MAE", mean_absolute_error(preds, y_test))
print("MAPE", mean_absolute_percentage_error(preds, y_test))
print("RMSE", np.sqrt(mean_squared_error(preds, y_test)))
get_feature_importances(clf)

# evaluate_errors(preds, y_test)

In [ ]:
import lightgbm as lgb
num_round = 10
param = {'num_leaves':31, 'num_trees':100, 'objective':'binary'}
param['metric'] = ['auc', 'binary_logloss']
train_data = lgb.Dataset(X_train, label=y_train)
test_data = []
test_data = lgb.Dataset(test_data, reference=train_data)
# test_data = lgb.Dataset(y_train)
bst = lgb.train(param, train_data, num_round, valid_sets=[test_data])

In [ ]:
import lightgbm as lgb

lgbmmodel = lgb.LGBMModel(boosting_type='gbdt', num_leaves=31, max_depth=-1,
                          learning_rate=0.1, n_estimators=1000, subsample_for_bin=200000,
                          objective='multiclass', class_weight='balanced', min_split_gain=0.0,
                          min_child_weight=0.001, min_child_samples=20, subsample=1.0,
                          subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0,
                          random_state=None, n_jobs=-1, silent=True, importance_type='split')

lgbmmodel.fit(X_train, y_train, feature_name=['high','medium','low'], categorical_feature=[0,1,2])
preds = lgbmmodel.predict(X_test)
print("MAE", mean_absolute_error(preds, y_test))
print("MAPE", mean_absolute_percentage_error(preds, y_test))
print("RMSE", np.sqrt(mean_squared_error(preds, y_test)))

In [ ]:
lgbmmodel = lgb.LGBMModel(boosting_type='gbdt', num_leaves=31, max_depth=-1,
                          learning_rate=0.1, n_estimators=1000, subsample_for_bin=200000,
                          objective='', class_weight=None, min_split_gain=0.0,
                          min_child_weight=0.001, min_child_samples=20, subsample=1.0,
                          subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0,
                          random_state=None, n_jobs=-1, silent=True, importance_type='split')

lgbmmodel.fit(X_train, y_train)
preds = lgbmmodel.predict(X_test)
print("MAE", mean_absolute_error(preds, y_test))
print("MAPE", mean_absolute_percentage_error(preds, y_test))
print("RMSE", np.sqrt(mean_squared_error(preds, y_test)))

In [ ]:
get_feature_importances(lgbmmodel)

In [ ]:
lgbmmodel.dump_model()

In [ ]:
preds

In [ ]:
y_train_dict['V000016']

In [ ]:
# CREATE AN ARRAY OF POTENTIAL CLASSIFIERS TO TEST THE PERFORMANCE OF EACH AND COMPARE

from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from sklearn.neighbors import KNeighborsRegressor

GRAD_BOOST_PARAMS = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

classifiers = [
#     MLPRegressor(solver='lbfgs', alpha=1e-5,
#                      hidden_layer_sizes=(5, 2), random_state=1),
#     RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1, n_jobs=-1),
#     linear_model.SGDRegressor(),
#     linear_model.BayesianRidge(),
#     linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.PassiveAggressiveRegressor(),
#     linear_model.TheilSenRegressor(),
#     linear_model.LinearRegression(),
#     LGBMRegressor(n_estimators=1000, learning_rate=0.01),
#     svm.SVR(kernel="linear"),
#     linear_model.ARDRegression(),
#     ensemble.GradientBoostingRegressor(**GRAD_BOOST_PARAMS)
    KNeighborsRegressor(n_neighbors=11, weights='uniform',
                        algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=-1)
]

In [ ]:
# RUN THE CLASSIFIERS AND PREDICT ON TEST DATA

for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
#     errors = evaluate_errors(preds, y_test)
    print("MAE", mean_absolute_error(preds, y_test))
    print("MAPE", mean_absolute_percentage_error(preds, y_test))
    print("RMSE", np.sqrt(mean_squared_error(preds, y_test)))
    try:
        get_feature_importances(clf)
    except:
        print("NO FEATURE IMPORTANCE METRIC")

In [ ]:
# THIS MODULE WILL OUTPUT A DICTIONARY WITH THE FEATURE RANKING OF A CLASSIFIER

from sklearn.feature_selection import RFECV
def RFEVC_(clf, X_train_, y_train_, step_=1, verbose_=0):
    selector = RFECV(clf, step=step_, verbose=verbose_, n_jobs=-1)
    selector = selector.fit(X_train_, y_train_)
    
    INPUT_COLS = X_train.columns
    RANKS = selector.ranking_

    rank_dict = {}
    for idx, item in enumerate(RANKS):
        rank_dict[INPUT_COLS[idx]] = item
    
    import pprint
    pprint.pprint(rank_dict)
    
    return rank_dict, selector

# USAGE EXAMPLE:
# rank_dict, selec = RFEVC_(regr, X_train, y_train)

In [ ]:
# RUN THE RECURSIVE FEATURE ELIMINATION ON EACH CLASSIFIER IN THE ARRAY

for item in classifiers:
    print(item)
    clf = item
    rank_dict, selector = RFEVC_(clf, X_train, y_train)

In [ ]:
import random

NUM_VARIETIES = 174

def best_yield_variety(regr, test_set, random_sel = True, n_samples = 174, print_variety_preds = True):
    
    #create empty df
    dup_df = pd.DataFrame()
    
    #choose a rand sample of input test_set (for dev purposes, wouldn't be used in app)
    test_set_sample = test_set.sample(n= n_samples) if random_sel else test_set
    
    #progress of intensive, long for loop upcoming
    counter = 0
    
    #for loop will, for each row in test_set_sample, duplicate that row by NUM_VARIETIES
    for index, row in test_set_sample.iterrows():
        counter+=1
        print(counter)
        dup_df = dup_df.append([row] * NUM_VARIETIES, ignore_index = True)
        
    #extract the varieties columns
    duplicated_df_varieties = dup_df.loc[:, dup_df.columns.str.match('V\d\d\d\d\d\d')]
    #extract the names of the varieties
    varieties_array = duplicated_df_varieties.columns
    num_expanded_data_pts = duplicated_df_varieties.shape[0]
    #we must have a zeroed matrix of the same shape as the duplicated_df_varieties
    #so that we can input a 1 just once for each variety per 174 rows
    d = np.zeros((duplicated_df_varieties.shape[0], NUM_VARIETIES))
    #make d our dataframe, with columns equal to the varieties
    duplicated_df_varieties = pd.DataFrame(d, columns=varieties_array)
    #for loop will place a 1 just once for each variety per 174 rows (one hot rep)
    for i in range(duplicated_df_varieties.shape[0]):
        var_index = i % 174
        duplicated_df_varieties.loc[i, varieties_array[var_index]] = 1
    #remove the varieties (will be added back with the new values)
    dup_df = dup_df.drop(varieties_array, axis = 1)
    #add the new values (one hot representations) from above for loop
    dup_df = pd.concat([dup_df, duplicated_df_varieties], axis=1)
    #do prediction on the entire dataframe
    preds_per_variety = regr.predict(dup_df)
    #*******make it into a dataframe where each row will give the performance of each variety
    #with the same environmental conditions*******
    preds_df = pd.DataFrame(preds_per_variety.reshape((int(num_expanded_data_pts/NUM_VARIETIES), NUM_VARIETIES)),
                            columns=varieties_array)
    
    #environmental conditions (everything except the variety data)
    envcond = test_set_sample.drop(varieties_array, axis=1)
    
    #a simple print out for best variety given the environmental conditions
    hr_preds = []
    if print_variety_preds:
        envcond_cols = envcond.columns
        counter = -1
        for idx, row in envcond.iterrows():
            counter+=1
            out = "For environmental conditions:\n%s\nthe best variety is:%s" % (row, preds_df.idxmax(axis=1)[counter])
            hr_preds.append(out)
            print(out)
    
    return preds_df, envcond, hr_preds
    
        
preds_df, envcond, hr_preds = best_yield_variety(regr, X_test, n_samples = 500)

In [ ]:
#find number of times a variety is the maximum in the above prediction dataframe
maxes = preds_df.idxmax(axis=1) # get first max variety per env
# print(pd.get_dummies(maxes).describe())
print(pd.get_dummies(maxes).sum().sort_values(ascending=False))

In [ ]:
#find number of times a variety is the maximum in the above prediction dataframe
maxes = preds_df.idxmax(axis=1) # get first max variety per env
# print(pd.get_dummies(maxes).describe())
print(pd.get_dummies(maxes).sum().sort_values(ascending=False))
print(len(pd.get_dummies(maxes).sum()))

In [ ]:
preds_df.plot(kind='line', figsize=(20,20))
# print(preds_df.loc[0].sort_values())
# type(preds_df.loc[0])

In [ ]:
preds_df_samples = preds_df.sample(10)

In [ ]:
preds_df_samples.plot(kind="area", figsize=(20,20))

In [ ]:
preds_df.describe().sort_values(by="mean", axis=1)

In [ ]:
preds_df.describe().loc['mean',:].sort_values().describe()

In [ ]:
# CLASSIFICATION METHODS BELOW!

In [ ]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=20, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))